# OBJECTIVE

> ### GOAL: **Save the contents of an `ihelp_menu2` widget in a form that can be used in either a Markdown or HTML document.**

- **ihelp_menu2 (in the functions.py file) accepts a list of functions to display both/either the help printout and/or source code for the list of functions.** 
    - The list of functions is presented as a Dropdown Menu with checkboxes for selecting which pieces of info to display.<br><br>

    - It uses a combination of `io.StringIO`, `sys.stdout`, and the `inspect.getsource` modules/functions to save a dictionary of the text results for each function, with the interactive menu controlling which function is *displayed*.
    
        - Note: this is different than how i originally did things with `ihelp_menu`, in which the documentationw as *generated* when the buttons were clicked. 
    

## Reference

- [FULL ipywidgets documentation](https://ipywidgets.readthedocs.io/en/latest/)
    - [RELATED SECTION: "Embedding Jupyer Notebook Widgets in Other Contexts"](https://ipywidgets.readthedocs.io/en/latest/embedding.html)
    
    
- **JAVASCRIPT SUBSECTIONS:**
    1. ['Embeddable HTML Snippet'](https://ipywidgets.readthedocs.io/en/latest/embedding.html#embeddable-html-snippet)
    2. ["Using `jupyer-widgets-controls` in web contexts"](https://ipywidgets.readthedocs.io/en/latest/embedding.html#using-jupyter-widgets-controls-in-web-contexts)
    
## What I've Tried:

1. "Using the Widgets> "Embed Widgets" from the J.Notebook menu bar. 
    - Issue: the contents were too big to copy and paste and kept crashing my browser.
2. "Using `ipywidgets.embed.embed_minimal_html`:
    - Creates a standalone web page which has my interface, but does not actually display the outputs.


In [1]:
from bs_ds.imports import *
import bs_ds as bs
from bs_ds import reload

import ipywidgets as widgets

## Function with widget display to save
import functions as ji

# from functions import ihelp_menu2
# reload(ji)

bs_ds  v0.9.10 loaded.  Read the docs: https://bs-ds.readthedocs.io/en/latest/index.html
For convenient loading of standard modules use: from bs_ds.imports import *



Package,Handle,Description
bs_ds,bs,Custom data science bootcamp student package
matplotlib,mpl,Matplotlib's base OOP module with formatting artists
matplotlib.pyplot,plt,Matplotlib's matlab-like plotting module
numpy,np,scientific computing with Python
pandas,pd,High performance data structures and tools
seaborn,sns,High-level data visualization library based on matplotlib


### Testing Use of Output on its own

In [2]:
out = widgets.Output(layout={'border': '1px solid black'})
display(out)
with out:
    help(widgets.interactive_output)

Output(layout=Layout(border='1px solid black'))

In [ ]:
# reload(ji)
# function_list = [bs.list2df, bs.save_model_weights_params]
# ji.ihelp_menu2(function_list)

# Attempts to Export Widget

- thoughts: `embed.embed_minimal_html` may wind up working for me, but I think the fact that I have widgets inside containers inside other containers may be complicating the export.

- may try to use the **Flexbox layout** to have all widgets in one container
    - https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html#The-Flexbox-layout

In [3]:
reload(ji)
function_list = [bs.list2df, bs.save_model_weights_params]

menu = ji.ihelp_menu2(function_list, to_embed=True)

Reloading...



In [4]:
menu.children

(GridBox(box_style='warning', children=(VBox(children=(Label(value='iHelp Menu: View Help and/or Source Code'), HBox(children=(Label(value='Function Menu'), Dropdown(options=('list2df', 'save_model_weights_params'), value='list2df'), ToggleButton(value=False, description='Show/hide'))))), HBox(children=(Checkbox(value=True, description="Show 'help(func)'"), Checkbox(value=True, description='Show source code'), Checkbox(value=False, description='Show source filepath'))))),
 Output(layout=Layout(border='1px solid black')))

In [5]:
from ipywidgets import embed#embed_minimal_html, dependency_state

embed.embed_minimal_html('export.html',views=None)#, views=list(menu.children), state=embed.dependency_state(list(menu.children)))

### RE-WRITING MENU LAYOUT USING FLEXBOX

In [40]:
help(widgets.Layout)

Help on class Layout in module ipywidgets.widgets.widget_layout:

class Layout(ipywidgets.widgets.widget.Widget)
 |  Layout specification
 |  
 |  Defines a layout that can be expressed using CSS.  Supports a subset of
 |  https://developer.mozilla.org/en-US/docs/Web/CSS/Reference
 |  
 |  When a property is also accessible via a shorthand property, we only
 |  expose the shorthand.
 |  
 |  For example:
 |  - ``flex-grow``, ``flex-shrink`` and ``flex-basis`` are bound to ``flex``.
 |  - ``flex-wrap`` and ``flex-direction`` are bound to ``flex-flow``.
 |  - ``margin-[top/bottom/left/right]`` values are bound to ``margin``, etc.
 |  
 |  Method resolution order:
 |      Layout
 |      ipywidgets.widgets.widget.Widget
 |      ipywidgets.widgets.widget.LoggingHasTraits
 |      traitlets.traitlets.HasTraits
 |      traitlets.traitlets.HasDescriptors
 |      builtins.object
 |  
 |  Data descriptors defined here:
 |  
 |  align_content
 |      An enum of strings where the case should be ign

In [80]:
def make_function_help_dict(function_list,json_file='ihelp_output.txt'):
    """Helper function for ihelp_menu which retreives all of the help and sourcecode text
    for each function in the function list and saves them to a json file with specified name.
    Returns output_dict with keys=function names, then dict[func_name]['help'] and 
    dict[func_name]['source'] anddict[func_name]['file_location']"""
    import pandas as pd
    import sys
    import inspect
    from io import StringIO
    notebook_output = sys.stdout
    result = StringIO()
    sys.stdout=result
    
    ## Turn single input into a list
    if isinstance(function_list,list)==False:
        function_list = [function_list]
    
    ## Make a dictionary of{function_name : function_object}
    functions_dict = dict()
    for fun in function_list:
        
        ## if input is a string, save string as key, and eval(function) as value
        if isinstance(fun, str):
            functions_dict[fun] = eval(fun)

        ## if input is a function, get the name of function using inspect and make key, function as value
        elif inspect.isfunction(fun):

            members= inspect.getmembers(fun)
            member_df = pd.DataFrame(members,columns=['param','values']).set_index('param')

            fun_name = member_df.loc['__name__'].values[0]
            functions_dict[fun_name] = fun
            
            
    ## Create an output dict to store results for functions
    output_dict = {}

    for fun_name, real_func in functions_dict.items():
        
        output_dict[fun_name] = {}
                
        ## First save help
        help(real_func)
        output_dict[fun_name]['help'] = result.getvalue()
        
        ## Clear contents of io stream
        result.truncate(0)
        
        try:
            ## Next save source
            print(inspect.getsource(real_func)) #eval(fun)))###f"{eval(fun)}"))
        except:
            print("Source code for object was not found")
        output_dict[fun_name]['source'] = result.getvalue()
        
        ## clear contents of io stream
        result.truncate(0)
        
        
        ## Get file location
        try:
            file_loc = inspect.getfile(real_func)
            print(file_loc)
        except:
            print("File location not found")
            
        output_dict[fun_name]['file_location'] =result.getvalue()
        
        
        ## clear contents of io stream
        result.truncate(0)        
                
        
    ## Reset display back to notebook
    sys.stdout = notebook_output    

    
    with open(json_file,'w') as f:
        import json
        json.dump(output_dict,f)

    return output_dict



def ihelp_menu3(function_list, json_file='ihelp_output.txt',to_embed=False):
    """Accepts a list of string names for loaded modules/functions to save the `help` output and 
    inspect.getsource() outputs to dictionary for later reference and display"""
    ## One way using sys to write txt file
    output_dict = make_function_help_dict(function_list=function_list, json_file=json_file)
    
    ## CREATE INTERACTIVE MENU
    from ipywidgets import interact, interactive, interactive_output
    import ipywidgets as widgets
    from IPython.display import display
    
    
    ## dropdown menu (dropdown, label, button)
    label = widgets.Label('Function Menu',
                          layout=widgets.Layout(flex='1 1 auto', 
                                                width='15%'))
    
    dropdown = widgets.Dropdown(options=list(output_dict.keys()),
                                layout=widgets.Layout(flex='3 1 auto', width='40%'))

    button = widgets.ToggleButton(description='Show/hide',value=False,
                                  layout=widgets.Layout(flex='1 1 auto',width='20%'),
                                 button_style='danger')
    
    ## saving widgets to one list
    items_auto = [label, dropdown, button]
    
    

    ## Check boxes
    check_help = widgets.Checkbox(description="Show 'help(func)'",value=True,
                                 layout=widgets.Layout(flex='1 1 auto', width='25%'))
    
    check_source = widgets.Checkbox(description="Show source code",value=True,
                                layout=widgets.Layout(flex='1 1 auto', width='25%'))
    
    check_fileloc=widgets.Checkbox(description="Show source filepath",value=False,
                                   layout=widgets.Layout(flex='1 1 auto', width='25%'))
    
    [items_auto.append(x) for x in [check_help, check_source, check_fileloc]]
    
    
        
#     ## Define output manager
    out = widgets.Output(layout=widgets.Layout(flex='1 1 auto',width='100%',border='1px solid black'))#{'border': '1px solid black'}
    items_auto.append(out)
    
    box_layout = widgets.Layout(display='flex-wrap',flex_flow='wrap',#'row',
                               align_items='stretch',width='90%')
    box_auto = widgets.Box(children=items_auto, layout=box_layout)
    
    final_layout = box_auto
    
    display(final_layout)
    
    
    def dropdown_event(change):
        print('Dropdown menu changed') 
        new_key = change.new
        output_display = output_dict[new_key]
    dropdown.observe(dropdown_event,names='values')

    
    def show_ihelp(display_help=button.value,function=dropdown.value,
                   show_help=check_help.value,show_code=check_source.value, show_file=check_fileloc.value, out=out):
                   #ouput_dict=output_dict):

        from IPython.display import Markdown
        from IPython.display import display        
        page_header = '---'*28
        import json
        with open(json_file,'r') as f:
            output_dict = json.load(f)
        
        
        func_dict = output_dict[function]

        with out:
            out.clear_output()

            if display_help:
                if show_help:
    #                 display(print(func_dict['help']))
                    print(page_header)
                    banner = ''.join(["---"*2,' HELP ',"---"*24,'\n'])
                    print(banner)
                    print(func_dict['help'])

                if show_code:
                    print(page_header)

                    banner = ''.join(["---"*2,' SOURCE -',"---"*23])
                    print(banner)
                    source_code = "```python\n"
                    source_code += func_dict['source']
                    source_code += "```"
                    display(Markdown(source_code))
                
                
                if show_file:
                    print(page_header)
                    banner = ''.join(["---"*2,' FILE LOCATION ',"---"*21])
                    print(banner)
                    
                    file_loc = func_dict['file_location']
                    print(file_loc)
                    
                # if show_help==False & show_code==False & show_file==False:
                #     display('Check at least one "Show" checkbox for output.')
                    
            # else:
                # display('Press "Show/hide" for display')
            else:
                out.clear_output()
            

    
    output = widgets.interactive_output(show_ihelp,{'display_help':button,
                                                   'function':dropdown,
                                                   'show_help':check_help,
                                                   'show_code':check_source,
                                                   'show_file':check_fileloc})

#     with output:
#         display(final_layout)#, output)
#         # display(output)
    
    if to_embed:
        return final_layout#,output #, output#final_layout



In [81]:
function_list = [bs.ihelp,bs.list2df, bs.save_model_weights_params]

menu = ihelp_menu3(function_list, to_embed=True)

Box(children=(Label(value='Function Menu', layout=Layout(flex='1 1 auto', width='15%')), Dropdown(layout=Layou…

In [86]:
from ipywidgets import embed #embed_minimal_html, dependency_state

embed.embed_minimal_html('export.html',views=list(menu.children))#, state=embed.dependency_state(list(menu.children)))

In [85]:
import json
menu_parts = list(menu.children)
data = embed.embed_data(views=menu_parts)

html_template = """
<html>
  <head>

    <title>Widget export</title>

    <!-- Load RequireJS, used by the IPywidgets for dependency management -->
    <script 
      src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.4/require.min.js" 
      integrity="sha256-Ae2Vz/4ePdIu6ZyI/5ZGsYnb+m0JlOmKPjt6XZ9JJkA=" 
      crossorigin="anonymous">
    </script>

    <!-- Load IPywidgets bundle for embedding. -->
    <script
      data-jupyter-widgets-cdn="https://cdn.jsdelivr.net/npm/"
      src="https://unpkg.com/@jupyter-widgets/html-manager@*/dist/embed-amd.js" 
      crossorigin="anonymous">
    </script>

    <!-- The state of all the widget models on the page -->
    <script type="application/vnd.jupyter.widget-state+json">
      {manager_state}
    </script>
  </head>

  <body>

    <h1>Widget export</h1>

    <div id="first-slider-widget">
      <!-- This script tag will be replaced by the view's DOM tree -->
      <script type="application/vnd.jupyter.widget-view+json">
        {widget_views[0]}
      </script>
    </div>

    <hrule />

    <div id="second-slider-widget">
      <!-- This script tag will be replaced by the view's DOM tree -->
      <script type="application/vnd.jupyter.widget-view+json">
        {widget_views[1]}
      </script>
    </div>

  </body>
</html>
"""

manager_state = json.dumps(data['manager_state'])
widget_views = json.dumps([json.dumps(view) for view in data['view_specs']])
rendered_template = html_template.format(manager_state=manager_state, widget_views=widget_views)

with open('export.html', 'w') as fp:
    fp.write(rendered_template)
   

## Using `embed.embed_data` (Have yet to fully try)


- [From ipywidgets docs: Embedding HTML> Python Interface](https://ipywidgets.readthedocs.io/en/latest/embedding.html#python-interface)


```python
 
```

In [ ]:
help(menu)

In [ ]:
import json

from ipywidgets import IntSlider
from ipywidgets.embed import embed_data

# s1 = IntSlider(max=200, value=100)
# s2 = IntSlider(value=40)

data = embed_data(views=[menu])

html_template = """
<html>
  <head>

    <title>Widget export</title>

    <!-- Load RequireJS, used by the IPywidgets for dependency management -->
    <script 
      src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.4/require.min.js" 
      integrity="sha256-Ae2Vz/4ePdIu6ZyI/5ZGsYnb+m0JlOmKPjt6XZ9JJkA=" 
      crossorigin="anonymous">
    </script>

    <!-- Load IPywidgets bundle for embedding. -->
    <script
      data-jupyter-widgets-cdn="https://cdn.jsdelivr.net/npm/"
      src="https://unpkg.com/@jupyter-widgets/html-manager@*/dist/embed-amd.js" 
      crossorigin="anonymous">
    </script>

    <!-- The state of all the widget models on the page -->
    <script type="application/vnd.jupyter.widget-state+json">
      {manager_state}
    </script>
  </head>

  <body>

    <h1>Widget export</h1>

    <div id="first-slider-widget">
      <!-- This script tag will be replaced by the view's DOM tree -->
      <script type="application/vnd.jupyter.widget-view+json">
        {widget_views[0]}
      </script>
    </div>

    <hrule />

    <div id="second-slider-widget">
      <!-- This script tag will be replaced by the view's DOM tree -->
      <script type="application/vnd.jupyter.widget-view+json">
        {widget_views[1]}
      </script>
    </div>

  </body>
</html>
"""

manager_state = json.dumps(data['manager_state'])
widget_views = json.dumps(data['view_specs'])#[json.dumps(view) for view in data['view_specs']]

In [ ]:
print(len(widget_views), len(manager_state))

In [ ]:
rendered_template = html_template.format(manager_state=manager_state, widget_views=widget_views)
with open('export.html', 'w') as fp:
    fp.write(rendered_template)
   

# For Easy Reference: ihelp_menu2 code

In [ ]:
bs.ihelp(ihelp_menu2)